# Prepare Data for Keras

There are 3 types of data:
**Nominal category** - a group of objects that can be collectively grouped on the basis of a particular characteristics -a qualitative property
**Ordinal data** - is a categorical data type where the variables have natural orderd categories and the distances between the categories is not known
**Numerical data**


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
from tqdm import tqdm

%run utils.ipynb
,.